In [50]:
# https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
import json
import numpy as np
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers.merge import concatenate

from keras.preprocessing import sequence
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [59]:
def getNonSeqData(data):
    output = []

    output.append(data['FurLength'])
    output.append(data['Color1'])
    
#     output.append(0 if data['Fee'] == 0 else 1)
    output.append(data['Fee'])
    
    output.append(data['Vaccinated'])
    output.append(data['Dewormed'])
    output.append(data['Sterilized'])
    output.append(data['MaturitySize'])
    
#     output.append(1 if data['Quantity'] == 1 else 2)
#     output.append(int(data['PhotoAmt']//5))
    output.append(data['Quantity'])
    output.append(data['PhotoAmt'])
    
    output.append(data['Gender'])
    
#     output.append(data['Age']//12)
    output.append(data['Age'])

    output.append(0 if data['Breed2'] == 0 else 1)

    return output

In [60]:
# load dataset
filename = 'text_and_label_all/json_vectorized_categorical.json'

X = []
y = []
with open(filename) as json_file:
    data = json.load(json_file)
    
    for key, value in data.items():
        X.append(value["vectorized"][:-12] + getNonSeqData(value))
        y.append(value["label"])

X = np.array(X)
y = np.array(y)
        
y = to_categorical(y, num_classes=5)
        
print(X[0])
print(y[0])

[12, 0, 0, 9, 0, 6, 15, 0, 0, 0, 8, 18, 20, 4, 0, 0, 0, 19, 0, 0, 17, 0, 1, 24, 7, 0, 23, 16, 0, 13, 14, 21, 22, 5, 2, 2, 0, 1, 1, 1, 2, 1, 2.0, 2, 3, 1]
[0. 0. 1. 0. 0.]


In [61]:
top_words = 5000
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [62]:
# truncate and pad input sequences
input_len = 512
X_train = sequence.pad_sequences(X_train, maxlen=input_len)
X_test = sequence.pad_sequences(X_test, maxlen=input_len)

X_train_seq = X_train[:, :-12]
X_train_other = X_train[:, -12:]

X_test_seq = X_test[:, :-12]
X_test_other = X_test[:, -12:]

print(X_train_seq.shape, X_train_other.shape, y_train.shape)
print(X_test_seq.shape, X_test_other.shape, y_test.shape)

(11447, 500) (11447, 12) (11447, 5)
(2862, 500) (2862, 12) (2862, 5)


In [63]:
embedding_vecor_length = 32
seq_len = 500

# seq data input
visible1 = Input(shape=(seq_len,))
embedding1 = Embedding(top_words, embedding_vecor_length, input_length=seq_len)(visible1)
conv1 = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(embedding1)
pool1 = MaxPooling1D(pool_size=2)(conv1)
lstm1 = LSTM(100)(pool1)

# non-seq data input
visible2 = Input(shape=(12,))
dense1 = Dense(20, activation='relu')(visible2)
dense2 = Dense(20, activation='relu')(dense1)

# merge input models
merge1 = concatenate([lstm1, dense2])

output = Dense(5, activation='softmax')(merge1)
model = Model(inputs=[visible1, visible2], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, 500)          0                                            
__________________________________________________________________________________________________
embedding_16 (Embedding)        (None, 500, 32)      160000      input_19[0][0]                   
__________________________________________________________________________________________________
conv1d_11 (Conv1D)              (None, 500, 32)      3104        embedding_16[0][0]               
__________________________________________________________________________________________________
input_20 (InputLayer)           (None, 12)           0                                            
__________________________________________________________________________________________________
max_poolin

In [66]:
model.fit([X_train_seq, X_train_other], y_train, epochs=5, batch_size=16)
# Final evaluation of the model
scores = model.evaluate([X_test_seq, X_test_other], y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/5
11447/11447 [==============================] - 151s 13ms/step - loss: 1.5445 - acc: 0.3210
Epoch 2/5
11447/11447 [==============================] - 150s 13ms/step - loss: 1.3927 - acc: 0.3869
Epoch 3/5
11447/11447 [==============================] - 147s 13ms/step - loss: 1.2997 - acc: 0.4447
Epoch 4/5
11447/11447 [==============================] - 145s 13ms/step - loss: 1.1795 - acc: 0.5022
Epoch 5/5
11447/11447 [==============================] - 145s 13ms/step - loss: 1.0697 - acc: 0.5579
Accuracy: 36.20%


In [67]:
# serialize model to JSON
model_json = model.to_json()
with open("models/multi_input_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("models/multi_input_model.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
from matplotlib import pyplot

history = model.fit([X_train_seq, X_train_other], y_train, epochs=5, batch_size=16, validation_data=([X_test_seq, X_test_other], y_test))

pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

Train on 11447 samples, validate on 2862 samples
Epoch 1/5
11447/11447 [==============================] - 147s 13ms/step - loss: 0.9177 - acc: 0.6344 - val_loss: 1.7459 - val_acc: 0.3421
Epoch 2/5
11447/11447 [==============================] - 150s 13ms/step - loss: 0.8025 - acc: 0.6831 - val_loss: 1.8933 - val_acc: 0.3543
Epoch 3/5
11447/11447 [==============================] - 151s 13ms/step - loss: 0.6932 - acc: 0.7306 - val_loss: 2.0439 - val_acc: 0.3532
Epoch 4/5
11447/11447 [==============================] - 2293s 200ms/step - loss: 0.6009 - acc: 0.7707 - val_loss: 2.2066 - val_acc: 0.3532
Epoch 5/5
  528/11447 [>.............................] - ETA: 2:13 - loss: 0.4844 - acc: 0.8087